In [1]:
import minsearch
import json

In [2]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

# Flatten the documents
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"] # SELECT * WHERE course = 'data-engineering-zoomcamp'; #kind of SQL query for keyword_fields
)

In [6]:
q = "the course has already started, can i still enroll?" #question example

In [7]:
index.fit(documents)

In [8]:
# All these fields como from elastic search and buy default they are all equal to 1.0

boost = {
    "question": 3.0, #the question field is 3 times mr important than the other fields
    'section':0.5 # this filed is 0.5 times less important than the other fields
}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'}, #filter for a specific course
    boost_dict=boost,
    num_results=5
)

In [9]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [10]:
from openai import OpenAI
import os

In [11]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [12]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": "is it too late to join the course?"
         },
    ]
)

In [13]:
response.choices[0].message.content

"Whether it’s too late to join a course depends on several factors, including the specific academic institution, the course structure, and the point in the academic calendar. Here are some steps you can take to find out if you can still join:\n\n1. **Check Enrollment Deadlines:** Review the enrollment deadlines on the educational institution's website or in the course catalog.\n\n2. **Contact the Instructor or Administration:** Reach out to the course instructor or academic advisor to inquire about the possibility of late enrollment. They may be able to provide an exception or accommodate you.\n\n3. **Assess Course Progress:** If the course has already started, consider the amount of material you’ve missed and whether you can realistically catch up.\n\n4. **Financial Considerations:** Be aware of any financial implications for late enrollment, such as late fees or adjusted tuition costs.\n\n5. **Course Format:** Some online courses or programs may have rolling admissions or flexible st

In [14]:
prompt_template = """
You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: {question}

CONTEXT:{context}

""".strip()

In [15]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [16]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [17]:
prompt = prompt_template.format(question=q, context=context).strip()

In [18]:
print(prompt)

You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: the course has already started, can i still enroll?

CONTEXT:section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your f

In [19]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": prompt
         },
    ]
)

In [20]:
print(response.choices[0].message.content)

Yes, even if you don't register initially, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.


## MORE MODULAR

In [21]:
def search(query):
    boost = {
        "question": 3.0, #the question field is 3 times mr important than the other fields
        'section':0.5 # this filed is 0.5 times less important than the other fields
    }

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'}, #filter for a specific course
        boost_dict=boost,
        num_results=10
    )
    
    return results

In [22]:
# verify the search function
search("How do i run Kafta?")

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repository on your computer will make it easy for you to access the instructors’ code and make pull requests (if you want to add your own notes or make changes to the course content).\nYou will probably also create your own repositories that host your notes, versions of your file, to do this. Here is a great tutorial tha

In [23]:
def build_prompt(query,search_results):
    
    prompt_template = """
    You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
    Use only the facts from CONTEXT when answering when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, out NONE

    QUESTION: {question}

    CONTEXT:{context}

    """.strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [24]:
def llm(prompt):
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    ) 
    
    return response.choices[0].message.content

In [25]:
query = "how do i run kafka?"
search_results = search(query)
prompt = build_prompt(query,search_results)
answer = llm(prompt)

In [26]:
print(answer)

NONE


In [27]:
query = "how do i run kafka?"

def rag(query):
    search_results = search(query) # we can replace this other search function (ex: elastic search)
    prompt = build_prompt(query,search_results) #we can optimize the prompt function
    answer = llm(prompt) #we can use another llm model (ex: mistral, ada, etc.)
    
    return answer

In [28]:
rag('The course has already started, can i still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

# Start Elasticsearch 8.4.3 (original with persistence)

```bash
docker run -it \
    --rm
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

#### the -rm flag is what is causing it not to persist

# You can use Docker volumes to persist Elasticsearch data outside of the container. 
# This way, data will survive container restarts and even when the container is removed.

## 1. Create a Docker volume named esdata:


```bash
docker volume create esdata
```

# 2. Modify your docker run command to mount this volume:

```bash
docker run -it \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    -v esdata:/usr/share/elasticsearch/data \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [29]:
#!curl http://localhost:9200

{
  "name" : "7997b24d1987",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "67QAFJvqR0WzZIN31SXHEw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [30]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [34]:
from elasticsearch import Elasticsearch

# Replace with appropriate host and port if necessary
es_client = Elasticsearch("http://localhost:9200")

In [35]:
# checking is it is working
es_client.info()

ObjectApiResponse({'name': '7997b24d1987', 'cluster_name': 'docker-cluster', 'cluster_uuid': '67QAFJvqR0WzZIN31SXHEw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [36]:
# this is kind of a SQL query, we are creating a new index, with the settings and mappings

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# very similar to minsearch, we are creating a new index, with the settings and mappings
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [37]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [38]:
from tqdm.auto import tqdm # this is a progress bar

/home/ruifspinto/.virtualenvs/llm-zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:18<00:00, 50.64it/s]


In [40]:
query = "I just discover the course, can I still enroll?"

In [41]:
search_query = {
        "size": 5, # number of results
        "query": {
            "bool": {
                "must": { # this is the query
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], #question is 3 times more important than the other fields
                        "type": "best_fields"
                    }
                },
                "filter": { # this is the filter
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

In [43]:
response = es_client.search(index=index_name, body=search_query)

In [44]:
results_docs = []

for hit in response['hits']['hits']:
    results_docs.append(hit['_source'])
    
results_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [45]:
## make it modular
def elastic_search(query):
    search_query = {
        "size": 5, # number of results
        "query": {
            "bool": {
                "must": { # this is the query
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], #question is 3 times more important than the other fields
                        "type": "best_fields"
                    }
                },
                "filter": { # this is the filter
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    results_docs = []

    for hit in response['hits']['hits']:
        results_docs.append(hit['_source'])
        
    return results_docs

In [46]:
def rag(query):
    search_results = elastic_search(query) # replaced search function with elastic_search
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [47]:
rag("The course has started, can i still join in ?")

"Yes, even if you don't register, you're still eligible to submit the homework. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."